In [1]:
import numpy as np
import pandas as pd
import math

import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pytorch_lightning import seed_everything
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


import dcor

from datetime import datetime

from torch import nn

import os 



[rank: 0] Global seed set to 42


In [2]:
# Create device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
n=1000
for model1 in range(1,4):
    for model2 in range(1,4):
        #if not os.path.exists("./results-BENN/result-" + str(model1) + "-" + str(model2) + "-" + str(n)): 
            #os.makedirs("./results-BENN/result-" + str(model1) + "-" + str(model2) + "-" + str(n)) 
        dcor_list=[]
        for t in range(1,101):
            x_train=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_train_" + str(t) + ".csv")
            x_train=x_train.drop('Unnamed: 0', axis=1)
            y_train=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_train_" + str(t) + ".csv")
            y_train=y_train.drop('Unnamed: 0', axis=1)
            x_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_test_" + str(t) + ".csv")
            x_test=x_test.drop('Unnamed: 0', axis=1)
            y_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_test_" + str(t) + ".csv")
            y_test=y_test.drop('Unnamed: 0', axis=1)
            z_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/z_test_" + str(t) + ".csv")
            z_test=z_test.drop('Unnamed: 0', axis=1)


            n=x_train.shape[0]
            p=x_train.shape[1]
            res_d=1
            m=2
            x_train = torch.tensor(x_train.values).to(torch.float)
            y_train = torch.tensor(y_train.values).to(torch.float)
            x_test = torch.tensor(x_test.values).to(torch.float)
            y_test = torch.tensor(y_test.values).to(torch.float)
            #print(x_train[:5],y_trans[:5])
            mse_loss = nn.MSELoss()
            # Build model
            class nn_dr_reg_model(nn.Module):
                def __init__(self, input_features, output_features, dim_red_features, hidden_units_d, hidden_units_e, dim_red_layers, ens_reg_layers):
                    super().__init__()
                    model_dim_red=[]
                    model_dim_red.append(nn.Linear(in_features=input_features, 
                                                out_features=hidden_units_d))
                    model_dim_red.append(nn.ReLU())
                    for i in range(1,dim_red_layers):
                        model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                    out_features=hidden_units_d))
                        model_dim_red.append(nn.ReLU())
                    model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                out_features=dim_red_features))
                    self.dim_red_layer_stack = nn.Sequential(*model_dim_red)

                    model_ens_reg=[]
                    model_ens_reg.append(nn.Linear(in_features=dim_red_features, out_features=hidden_units_e))
                    model_ens_reg.append(nn.ReLU())
                    for i in range(1,ens_reg_layers):
                        model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=hidden_units_e))
                        model_ens_reg.append(nn.ReLU())
                    model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=output_features))
                    self.ens_reg_layer_stack = nn.Sequential(*model_ens_reg)
                
                def forward(self, x):
                    suff_predictor = self.dim_red_layer_stack(x)
                    ens_output = self.ens_reg_layer_stack(suff_predictor)
                    return ens_output, suff_predictor
                    

            # Create an instance of BlobModel and send it to the target device
            model_nn = nn_dr_reg_model(input_features=p, 
                                    output_features=1, 
                                    dim_red_features=res_d, 
                                    hidden_units_d=200,
                                    hidden_units_e=200,
                                    dim_red_layers=4, 
                                    ens_reg_layers=4
                                    ).to(device)
            model_nn
            optimizer = torch.optim.Adam(model_nn.parameters(), 
                                        lr=0.001)
            epochs = 200
            x_train, y_train = x_train.to(device), y_train.to(device)
            x_test, y_test = x_test.to(device), y_test.to(device)
            for epoch in range(epochs):
                ### Training
                model_nn.train()

                # 1. Forward pass
                y_pred_train, y_suff_train = model_nn(x_train) 
                
                # 2. Calculate loss and accuracy
                loss = mse_loss(y_pred_train, y_train) 
                
                # 3. Optimizer zero grad
                optimizer.zero_grad()

                # 4. Loss backwards
                loss.backward()

                # 5. Optimizer step
                optimizer.step()

                ### Testing
                model_nn.eval()

                y_pred_test, y_suff_test = model_nn(x_test)
                loss_test = mse_loss(y_pred_test, y_test) 
                #dcor_test = dcor.distance_correlation(y_suff_test.detach().numpy(),z_test)

                if epoch % 25 == 0:
                    #print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test Loss: {loss_test:.5f} | Dcor: {dcor_test:.5f}")
                    now = datetime.now()
                    current_time = now.strftime("%H:%M:%S")
                    #print("Current Time =", current_time)
            model_nn.eval()
            with torch.inference_mode():
                y_pred_test, y_suff_test = model_nn(x_test)
            y_suff_test=y_suff_test.numpy()
            dcor_current=dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()))
            dcor_list.append(dcor_current)
            print(model1, model2, t, dcor_current)
        dcor_list_df=pd.DataFrame(dcor_list)
        dcor_list_df.to_csv("./results-BENN/result-" + str(model1) + "-" + str(model2) + "-" + str(n) + ".csv")
        print(model1, model2, np.mean(dcor_list), np.std(dcor_list))


1 1 1 0.8937894381631173
1 1 2 0.9036315078012284
1 1 3 0.8955612450028319
1 1 4 0.9064053803338077
1 1 5 0.9113781862474227
1 1 6 0.921937377925426
1 1 7 0.9272431702470322
1 1 8 0.9069060427255553
1 1 9 0.9210962330308234
1 1 10 0.9158083673389549
1 1 11 0.9144444587718185
1 1 12 0.9117958457468278
1 1 13 0.9239614892076943
1 1 14 0.9195901994408147
1 1 15 0.8663313709806211
1 1 16 0.9084815035512226
1 1 17 0.8959779166008492
1 1 18 0.829715891535429
1 1 19 0.8960279537802218
1 1 20 0.9178370311547425
1 1 21 0.9165593342332676
1 1 22 0.9139318612922469
1 1 23 0.9160673577262444
1 1 24 0.9172767488851804
1 1 25 0.8937086219656901
1 1 26 0.9095461185460672
1 1 27 0.9098764557024663
1 1 28 0.9051245286747874
1 1 29 0.8747219468840618
1 1 30 0.9156128441807695
1 1 31 0.9202364505978683
1 1 32 0.9054521643843559
1 1 33 0.912718416079912
1 1 34 0.9104584424286573
1 1 35 0.9140226075459078
1 1 36 0.9050946241871375
1 1 37 0.9064880625534688
1 1 38 0.9089278171511105
1 1 39 0.918661363227987

In [ ]:
#plt.plot(y_suff_test,z_test,"o")
#plt.show()

In [ ]:
#y_suff_test_df=pd.DataFrame(y_suff_test)
#y_pred_test_df=pd.DataFrame(y_pred_test)

In [ ]:
#dcor.distance_correlation(y_suff_test,z_test)

In [ ]:
#dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()))